In [1]:
import pandas as pd
import numpy as np
import rasterio
from rasterio import *
from rasterio.plot import show
from pyspatialml import Raster

from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE


In [2]:
predictors = pd.read_csv("out_data/forest_data_poland_lucas.csv", sep=" ",  index_col=False, na_values = 48,
                         header = None)
pd.set_option('display.max_columns',None)


Better variable names

In [3]:
predictors_names = pd.read_csv("out_data/available_features_names.csv", sep=",",  index_col=False)
predictors.columns = predictors_names.name.tolist()
predictors.tail(5)

,GLAD_mean100m,GLAD_median100m,GLAD_mode100m,GLAD_q1,GLAD_q3,CLC10m_mean,CLC10m_median,CLC10m_mode.,CLC10m_q1,CLC10m_q3,CLC_01.vrt,C-GLOPS_mean,C-GLOPS_median,C-GLOPS_mode,C-GLOPS_q1,C-GLOPS_q3,JAXA_q3,JAXA_q1,JAXA_mode,JAXA_median,JAXA_mean
23079,0.246136,0,0,0,1,0.60,1,1,0,1,0,0.191063,0.07,0.07,0.04,0.08,0.5,0.0,0.0,0.0,0.277736
23080,0.000000,0,0,0,0,0.00,0,0,0,0,0,0.016844,0.01,0.01,0.01,0.03,0.0,0.0,0.0,0.0,0.000000
23081,0.932173,1,1,1,1,0.95,1,1,1,1,1,0.402705,0.44,0.39,0.39,0.62,1.0,1.0,1.0,1.0,0.961699
23082,0.000000,0,0,0,0,0.00,0,0,0,0,0,0.043004,0.03,0.03,0.02,0.03,0.0,0.0,0.0,0.0,0.023998
23083,0.000000,0,0,0,0,0.09,0,0,0,0,0,0.098559,0.09,0.06,0.06,0.11,0.0,0.0,0.0,0.0,0.000000


Now the left hand side

In [4]:
outcome = pd.read_csv("out_data/LUCAS/LUCAS_locations.csv", sep=",",  index_col=False)
outcome.tail(5)

,X,Y,LC1,LC1_PERC,forest
23079,5.278000e+06,3.114000e+06,C22,100,1
23080,5.278000e+06,3.136000e+06,B11,100,0
23081,5.294000e+06,3.150000e+06,C10,100,1
23082,5.294000e+06,3.172000e+06,B11,92,0
23083,5.286000e+06,3.116000e+06,Bx1,98,0


In [5]:
print (len(outcome))
print (len(predictors))

23084
23084


In [6]:
outcome.head()

,X,Y,LC1,LC1_PERC,forest
0,5.214000e+06,3.064000e+06,B16,95,0
1,5.214000e+06,3.116000e+06,E20,100,0
2,5.214000e+06,3.388000e+06,C10,100,1
3,5.214000e+06,3.474000e+06,B16,1,0
4,4.792000e+06,3.082000e+06,B14,50,0


Split the Dataset

In [7]:
X    = predictors
Y    = outcome.iloc[:,4].to_frame()
feat = predictors.columns.values
print(X.shape)
print(Y.shape)
print(type(X))
print(type(Y))

(23084, 21)
(23084, 1)
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


There are a handful of NAs

In [8]:
na_rows = X.apply(lambda x: np.any(pd.isna(x)), axis=1)
result = np.where(na_rows)[0]
print('number NA: ' + str(len(result)))

number NA: 1265


In [9]:
X = X.drop(X.index[result])
Y = Y.drop(Y.index[result])


In [10]:
print(X.shape)
print(Y.shape)

(21819, 21)
(21819, 1)


Create 4 dataset for training and testing the algorithm 

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=2023)
y_train = np.ravel(Y_train)
y_test = np.ravel(Y_test)

In [12]:
print(np.mean(y_train==1))
print(np.mean(y_test==1))

0.3208360069667247
0.3242896425297892


Use wandb to seach the model space 

In [13]:
import wandb

In [14]:
my_entity="hofer1991"
my_project="RF_Foresthybrid_selection_config"

wandb.init(project=my_project)

wandb: Currently logged in as: hofer1991 (nodes). Use `wandb login --relogin` to force relogin


In [18]:
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'BIC/val',
      'goal': 'maximize'   
    },
    'parameters': {
        'rf__max_features': {
            'distribution': 'int_uniform',
            'min' : 3 ,
            'max' : 10
        },
        'rf__max_samples':{
            'distribution': 'uniform',
            'min' : 0.3 ,
            'max' : 0.7
        },
        'rf__n_estimators':{
            'distribution': 'int_uniform',
            'min' : 500 ,
            'max' : 2000
        },
        'rf__max_depth':{
            'distribution': 'int_uniform',
            'min' : 50 ,
            'max' : 300
        }
    }
}

sweep_config

{'method': 'bayes',
 'metric': {'name': 'BIC/val', 'goal': 'maximize'},
 'parameters': {'rf__max_features': {'distribution': 'int_uniform',
   'min': 3,
   'max': 10},
  'rf__max_samples': {'distribution': 'uniform', 'min': 0.3, 'max': 0.7},
  'rf__n_estimators': {'distribution': 'int_uniform', 'min': 500, 'max': 2000},
  'rf__max_depth': {'distribution': 'int_uniform', 'min': 50, 'max': 300}}}

We want to have the features as 0/1s as config parameters. 
For that we need to spell them out into the config dictionary. 
The next cell does this automatically for all features avaiable, so we don't have to write htem out. 

In [19]:
prior_model_size = 5

prior_inclusion_probability = prior_model_size / len(predictors.columns)

# Add parameters based on column names
for column_name in predictors.columns:
    sweep_config['parameters']["feat__" + column_name] = {
        'values': [0, 1],
        'probabilities': [1 - prior_inclusion_probability, prior_inclusion_probability]
    }


In [17]:
sweep_config

{'method': 'bayes',
 'metric': {'name': 'BIC/val', 'goal': 'maximize'},
 'parameters': {'rf__max_features': {'distribution': 'int_uniform',
   'min': 3,
   'max': 10},
  'rf__max_samples': {'distribution': 'uniform', 'min': 0.3, 'max': 0.7},
  'rf__n_estimators': {'distribution': 'int_uniform', 'min': 500, 'max': 2000},
  'rf__max_depth': {'distribution': 'int_uniform', 'min': 50, 'max': 300},
  'feat__GLAD_mean100m': {'values': [0, 1],
   'probabilities': [0.7619047619047619, 0.23809523809523808]},
  'feat__GLAD_median100m': {'values': [0, 1],
   'probabilities': [0.7619047619047619, 0.23809523809523808]},
  'feat__GLAD_mode100m': {'values': [0, 1],
   'probabilities': [0.7619047619047619, 0.23809523809523808]},
  'feat__GLAD_q1': {'values': [0, 1],
   'probabilities': [0.7619047619047619, 0.23809523809523808]},
  'feat__GLAD_q3': {'values': [0, 1],
   'probabilities': [0.7619047619047619, 0.23809523809523808]},
  'feat__CLC10m_mean': {'values': [0, 1],
   'probabilities': [0.76190476

In [20]:
predictors.columns

# Add parameters based on column names
for column_name in predictors.columns:
    sweep_config['parameters']["feat__" + column_name] = {'values': [0, 1]}


In [21]:
# Initialize a new sweep
# Arguments:
#     – sweep_config: the sweep config dictionary defined above
#     – entity: Set the username for the sweep
#     – project: Set the project name for the sweep
sweep_id = wandb.sweep(sweep_config, entity=my_entity, project=my_project)

Create sweep with ID: lwyo5elz
Sweep URL: https://wandb.ai/hofer1991/RF_Foresthybrid_selection_config/sweeps/lwyo5elz


In [22]:
# Training and Evaluation routines for Sweeping
def performance_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = (tp + tn) / (tn + tp + fp + fn)
#    tpr = tp / (tp + fn)
#    tnr = tn / (tn + fn)
#    bacc= (tpr + tnr) * 0.5
#    f1 = (tpr * tnr) / (tpr + tnr)
    return(acc)

def calculate_bic(y_true, y_pred, num_params):
    resid = y_true - y_pred
    sse = np.sum(resid**2)
    n = len(y_true)
    bic = num_params * np.log(n) + n * np.log(sse/n)
    return -bic

def evaluate_acc(model, X, y):
    y_pred = model.predict(X) 
    metric  = performance_metrics(y, y_pred)
    return(metric)
    

def evaluate_bic(model, X, y, num_params):
    y_pred = model.predict(X) 
    metric  = calculate_bic(y, y_pred, num_params )
    return(metric)

def build_select_index(config):
    print("builder started")
    select_index = []
    for key, value in config.items():
        if (key[:4] == 'feat'):
            select_index.append(value)
    return(select_index)


def train(config=None):

    with wandb.init(config=config):
        
        # Config is a variable that holds and saves hyperparameters and inputs
        config = wandb.config

        select_index = build_select_index(config)
        n_features = sum(select_index)
        
        selector = RandomForestClassifier(n_estimators=config.rf__n_estimators,
                              max_features=min(config.rf__max_features, n_features),
                              max_depth=config.rf__max_depth,
                              max_samples=config.rf__max_samples,
                              n_jobs=-1,
                              oob_score = True) 

        selector.fit(X_train, y_train)
  
        bic_train = evaluate_bic(selector,X_train, y_train, n_features)
        bic_test  = evaluate_bic(selector, X_test, y_test, n_features)
        
        acc_train = evaluate_acc(selector,X_train, y_train)
        acc_test  = evaluate_acc(selector, X_test, y_test)

        
        wandb.log({
               "BIC/train": bic_train,
               "BIC/val": bic_test,
                "Accuracy/train": acc_train,
                "Accuracy/val": acc_test, 
                "n_features": n_features
        })
        
    
        
        # Save the model with the best accuracy
        if bic_test < wandb.run.summary.get("best_accuracy", 0):
            print('bic_test: ',bic_test)
            wandb.run.summary["best_accuracy"] = bic_test
            model_name = "models/best_RF_Foresthybrid" + str(n_features) + "_featuresel.pkl"
            joblib.dump(selector, model_name)
    
        

In [23]:
wandb.agent(sweep_id, train)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: xaczho7h with config:
wandb: 	feat__C-GLOPS_mean: 0
wandb: 	feat__C-GLOPS_median: 0
wandb: 	feat__C-GLOPS_mode: 0
wandb: 	feat__C-GLOPS_q1: 0
wandb: 	feat__C-GLOPS_q3: 1
wandb: 	feat__CLC10m_mean: 0
wandb: 	feat__CLC10m_median: 0
wandb: 	feat__CLC10m_mode.: 1
wandb: 	feat__CLC10m_q1: 1
wandb: 	feat__CLC10m_q3: 0
wandb: 	feat__CLC_01.vrt: 0
wandb: 	feat__GLAD_mean100m: 0
wandb: 	feat__GLAD_median100m: 0
wandb: 	feat__GLAD_mode100m: 1
wandb: 	feat__GLAD_q1: 1
wandb: 	feat__GLAD_q3: 1
wandb: 	feat__JAXA_mean: 0
wandb: 	feat__JAXA_median: 0
wandb: 	feat__JAXA_mode: 0
wandb: 	feat__JAXA_q1: 1
wandb: 	feat__JAXA_q3: 1
wandb: 	rf__max_depth: 170
wandb: 	rf__max_features: 6
wandb: 	rf__max_samples: 0.6518957924688987
wandb: 	rf__n_estimators: 994


<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


builder started


Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


Accuracy/train,▁
Accuracy/val,▁
BIC/train,▁
BIC/val,▁
n_features,▁
Accuracy/train,0.99212
Accuracy/val,0.90018
BIC/train,52757.86592
BIC/val,25066.84224
n_features,8


...

wandb: Agent Starting Run: a738xtct with config:
wandb: 	feat__C-GLOPS_mean: 1
wandb: 	feat__C-GLOPS_median: 1
wandb: 	feat__C-GLOPS_mode: 1
wandb: 	feat__C-GLOPS_q1: 1
wandb: 	feat__C-GLOPS_q3: 1
wandb: 	feat__CLC10m_mean: 1
wandb: 	feat__CLC10m_median: 1
wandb: 	feat__CLC10m_mode.: 0
wandb: 	feat__CLC10m_q1: 0
wandb: 	feat__CLC10m_q3: 0
wandb: 	feat__CLC_01.vrt: 1
wandb: 	feat__GLAD_mean100m: 0
wandb: 	feat__GLAD_median100m: 1
wandb: 	feat__GLAD_mode100m: 1
wandb: 	feat__GLAD_q1: 0
wandb: 	feat__GLAD_q3: 1
wandb: 	feat__JAXA_mean: 0
wandb: 	feat__JAXA_median: 0
wandb: 	feat__JAXA_mode: 0
wandb: 	feat__JAXA_q1: 0
wandb: 	feat__JAXA_q3: 1
wandb: 	rf__max_depth: 171
wandb: 	rf__max_features: 4
wandb: 	rf__max_samples: 0.4030430660190135
wandb: 	rf__n_estimators: 1419
Widget Javascript not detected.  It may not be installed or enabled properly. Reconnecting the current kernel may help.


builder started


wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7f954a3613f0> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe